In [1]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import numpy as np
import pandas as pd

from quantfreedom.base import backtest_df_only
from quantfreedom.nb.simulate import simulate_up_to_6_nb
from quantfreedom.evaluators.evaluators import eval_is_below
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.testing import *
from quantfreedom.plotting.strat_dash import strat_dashboard

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

prices = pd.read_hdf("../data/prices.hd5")

rsi_ind = from_talib(
    func_name="rsi",
    prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=[15, 30],
)
entries = eval_is_below(
    want_to_evaluate=rsi_ind,
    user_args=[30, 40, 50],
    cand_ohlc="close",
)

In [2]:
strat_results_df, settings_results_df = backtest_df_only(
    prices=prices,
    entries=entries,
    equity=1000.0,
    fee_pct=0.06,
    mmr=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(1, 3.1, 1),
    risk_rewards=np.arange(1, 3.1, 1),
    size_pct=1.0,
    sl_pcts=np.arange(1,3.1,1),
)

Checking static variables for errors or conflicts.
Turning all variables into arrays.
Checking arrays for errors or conflicts ... the backtest will begin shortly, please hold.
Creating cartesian product ... after this the backtest will start I promise :).

Starting the backtest now ... and also here are some stats for your backtest.

Total symbols: 5
Total indicator settings per symbol: 6
Total indicator settings to test: 30
Total order settings per symbol: 27
Total order settings to test: 135
Total candles per symbol: 9,408
Total candles to test: 7,620,480

Total combinations to test: 810


In [ ]:
strat_results_df

In [ ]:
strat_results_df, settings_results_df = backtest_df_only(
    prices=prices,
    entries=entries,
    equity=1000.0,
    fee_pct=0.06,
    mmr=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(3, 7.1, 1),
    risk_rewards=np.arange(3, 11.1, .5),
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(2, 9.1, .5),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(2, 9.1, .5),
    tsl_when_pct_from_avg_entry=np.arange(2, 9.1, 1),
    gains_pct_filter=150,
)

In [ ]:
strat_results_df

In [ ]:
settings_results_df.iloc[:,1251]

In [ ]:
order_records = simulate_up_to_6_nb(
    prices=prices['BTCUSDT'].values,
    entries=entries.iloc[:,[5]].values,
    equity=1000.0,
    fee_pct=0.06,
    mmr=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=5,
    risk_rewards=2,
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_pcts_init=5,
    tsl_trail_by_pct=3,
    tsl_when_pct_from_avg_entry=3,
    # gains_pct_filter=100,
)

In [ ]:
entries[[("BTCUSDT", 30, 50)]]

In [ ]:
rsi_ind[[('BTCUSDT', 30)]]

In [ ]:
indicator_dict = {
    "indicator1": {
        "values1": rsi_ind[[('BTCUSDT', 30)]],
        "entries1": entries[[("BTCUSDT", 30, 50)]],
    },
}
strat_dashboard(
    indicator_dict=indicator_dict,
    prices=prices.BTCUSDT,
    order_records=order_records,
)